# Power Consumption Comparison

In [ ]:
# Imports

import matplotlib.pyplot as plt
import numpy as np
import requests
from dotenv import load_dotenv
import os

In [ ]:
# Shared variables
load_dotenv()

horreum_base = os.environ.get('HORREUM_BASE_URL')
base_api = horreum_base + "/api/run/"

In [ ]:
def fetch_label_values(run_id: str) -> dict:
    """
    Fetch the run's label values from the Horreum API.
    :param run_id: run identifier
    :return: the run's label values in dictionary format
    """
    label_values = requests.get(base_api + run_id + "/labelValues")
    return label_values.json()[0]["values"]

In [ ]:
# adopted tools to compare
tools = ['RAPL', 'Kepler', 'PSU', 'CPU']

# mapping from tool to label value name
tools_to_label = {
    'RAPL': 'power_rapl',
    'Kepler': 'power_kepler',
    'PSU': 'power_chasis',
    'CPU': 'cpu_pidstat',
}

In [ ]:
def plot(_run_ids: list[str], _tool: str, _save: bool = False) -> None:
    """
    Plot the power measurements obtained for a specific tool over time.
    :param _run_ids: Horreum run ids to compare power measurements for.
    :param _tool: see @tools.
    :param _save: if true, save the plot.
    """

    # It's good practice to create a figure and an axes object
    fig, ax = plt.subplots(figsize=(12, 6))

    for run_id in _run_ids:
        # --- Fetch data for the specific run ---
        run_values = fetch_label_values(run_id)
        injection_rate = run_values["InjectionRate"]
        input_data = run_values[tools_to_label.get(_tool)]
        # 1. Create a time axis
        # Since each sample is one second, the time axis is just 0, 1, 2, ...
        num_samples = len(input_data)
        time_axis = np.arange(num_samples) # This creates an array [0, 1, 2, ..., num_samples-1]

        # 2. Plot the data
        ax.plot(time_axis, input_data, label='RPS ' + str(injection_rate))

    ax.set_title(tool + ' Measurements Over Time', fontsize=16)
    ax.set_xlabel('Samples', fontsize=12)
    ax.set_ylabel('Watts', fontsize=12)

    ax.grid(True)
    ax.legend(fontsize=10)

    if _save:
        fig.savefig(_tool + '.png', dpi=300)

    plt.tight_layout()
    plt.show()

## Previous measurements

```dotenv
Java   = 21.0.5-tem
Kepler = v0.7.12
Cores  = All
```

In [ ]:
# runs to consider, each one uses a specific RPS
runs = ['66747', '66746', '66744', '66743', '66741']

# iterate over all tools and plot a chart with all RPS
for tool in tools:
    plot(runs, tool)

In [ ]:
# adopted tools to compare
tools = ['RAPL', 'Kepler', 'PSU', 'CPU', 'Kepler Node']

# mapping from tool to label value name
tools_to_label = {
    'RAPL': 'power_rapl',
    'Kepler': 'power_kepler',
    'PSU': 'power_chasis',
    'CPU': 'cpu_pidstat',
    'Kepler Node': 'power_kepler_node',
}

## New measurements

```dotenv
Java   = 21.0.5-tem
Kepler = v0.10.2
Cores  = 2
App    = Baremetal
```

In [ ]:
# runs to consider, each one uses a specific RPS
# '98778', '99078', 25k 30k
runs = ['99140', '99137', '99136', '99133', '99130']
for tool in tools:
    plot(runs, tool, False)

## New measurements

```dotenv
Java   = 21.0.5-tem
Kepler = v0.10.2
Cores  = 2
App    = Container
```

In [ ]:
runs = ['99156', '99153', '99159', '99149', '99146']
for tool in tools:
    plot(runs, tool, False)